In [41]:
import numpy as np
from random import randint
import pandas as pd
from random import shuffle

In [54]:
def weightedRandom(row, tpath):
    x = randint(0, (tpath*25))
    n = ((tpath*25)-row['fitness'])*1.2
    if x <= n:
        return 1
    else:
        return 0

def generatePopulation(df, path, iValDF, startPosition):
    shuffle(path)
    df.loc[-1] = [path, evalFit(iValDF, path, startPosition)]
    df = df.reset_index(drop=True)
    return df

def evalFit(df, path, startPosition):
    finalFitness = 0
    #path = path.values[0]
    for i, val in enumerate(path):
        if i < len(path)-1:
            currentRow = df.loc[(df['from_id'] == path[i]) & (df['to_id'] == path[i+1])]
            #print("CurrentRow print: ", currentRow['car_m_t'].tolist()[0])
            finalFitness += currentRow['car_m_t'].tolist()[0]
    currentRow = df.loc[(df['from_id'] == startPosition) & (df['to_id'] == path[len(path)-1])]
    finalFitness += currentRow['car_m_t'].tolist()[0]
    return finalFitness


def parentSelection(df, n, path):
    df['pass'] = df.apply(weightedRandom,tpath=len(path), axis=1)
    if(len(df['sequence']) > n):
        val = df.iloc[0].fitness
        df = df[~(df['fitness'] >= val+10)]
    #if(len(df) > 1):
        #df = df.drop_duplicates(subset=['sequence'])
    df = df.sort_values(by=['fitness']).reset_index(drop=True)
    return df

def selectPassed(df):
    df = df.loc[df['pass'] == 1]
    n1, n2 = randint(0, len(df['sequence'])-1), randint(0, len(df['sequence'])-1)
    pass1 = df['sequence'].iloc[n1]
    pass2 = df['sequence'].iloc[n2]
    return pass1, pass2

def crossover(df):
    ind1, ind2 = selectPassed(df)
    size = len(ind1)
    
    p1, p2 = [0]*size, [0]*size
    
    cxpoint1 = randint(0, size - 1)
    cxpoint2 = randint(0, size - 2)
    if cxpoint2 >= cxpoint1:
        cxpoint2 += 1
    else: # Swap the two cx points
        cxpoint1, cxpoint2 = cxpoint2, cxpoint1

    # Apply crossover between cx points
    for i in range(cxpoint1, cxpoint2):
        print(i)
        # Keep track of the selected values
        temp1 = ind1[i]
        temp2 = ind2[i]
        # Swap the matched value
        #[5787546, 5787549, 5789449, 5791360, 5791361, 5791356, 5793270, 5791353, 5785640]
        ind1[i], ind1[p1[temp2]] = temp2, temp1
        ind2[i], ind2[p2[temp1]] = temp1, temp2
        # Position bookkeeping
        p1[temp1], p1[temp2] = p1[temp2], p1[temp1]
        p2[temp1], p2[temp2] = p2[temp2], p2[temp1]
        
    df.loc[-1] = [ind1, evalFit(iValDF, ind1, startPosition)]
    df.loc[-2] = [ind2, evalFit(iValDF, ind2, startPosition)]
    df = df.reset_index(drop=True)
    return df

def mutation(df):
    
    df = df.reset_index(drop=True)
    return df

def GA(iValDF, path, startPosition):
    populationSize = 20
    resultsOverTimeList = []
    df = pd.DataFrame(columns=['sequence', 'fitness'])
    tempfit = 0
    df['sequence'] = [path]
    df['fitness'] = evalFit(iValDF, path, startPosition)
    for i in range(9):
        df = generatePopulation(df, path, iValDF, startPosition)
    df = parentSelection(df, populationSize, path)
    print("Starting iterations... Randomized population best fitness:")
    print(df.loc[0])
    
    for i in np.arange(100):
        df = crossover(df)
        df = parentSelection(df, populationSize,path)
        if i%100 == 0 and i != 0:
            if tempfit == df['fitness'].iloc[0]:
                df = mutation(df, iValDF)
            else:
                tempfit = df['fitness'].iloc[0]
            #print(i, " iterations completed. Current best fitness:")
            #print(df.loc[0])
        resultsOverTimeList.append(df['fitness'].loc[0])
        
    print("All iterations completed. Best fitness achieved:")
    print(df.loc[0])
    return df, resultsOverTimeList